# 마을방송 무선국 정보 API (단위 : MHz)
##### >> 사용자가 주소 정보를 통해 해당 지역에서 청취 가능한 마을방송 무선국 정보 및 주파수 정보를 제공 받을 수 있습니다.

### 데이터 받는 방법
##### 1. openAPI - X (전체 데이터를 받기가 어렵다)
##### https://spectrummap.kr/cop/bbs/selectBoardArticle.do?menuNo=300543&bbsId=BBSMSTR_000000000172&nttId=14708&menuNo=300543&gubun=1
##### 2. 웹에서 직접 받은 후 가공 - O (전체 데이터 받기위해 이 방법이 훨씬 좋다)
##### https://spectrummap.kr/gis/town_radiomap.do?menuNo=300511

In [1]:
# 패키지 임포트
import requests
import pandas as pd
import json
import re
import os

import warnings
warnings.filterwarnings('ignore')

In [3]:
# 경위도 좌표치환 메서드
def dms_to_decimal(degrees, minutes, seconds):
    return degrees + (minutes / 60) + (seconds / 3600)

# 문자열에서 도, 분, 초를 추출하는 함수
def parse_dms(dms_str):
    # 정규식을 이용해 DMS 문자열에서 도, 분, 초를 추출
    match = re.match(r"(\d+)° (\d+)' (\d+\.?\d*)\"", dms_str)
    if match:
        degrees = int(match.group(1))
        minutes = int(match.group(2))
        seconds = float(match.group(3))
        return dms_to_decimal(degrees, minutes, seconds)
    else:
        return None

### 1. openAPI 예시코드 (전체 데이터를 받을 수 없습니다. 웹에서 원본 데이터 받으세요)

In [30]:
# 마을방송 무선국 정보 API
url = 'https://spectrummap.kr/openapiNew.do?'

key = ''

# 요청 헤더에 User-Agent 추가
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

params = {
    'key': key,  # 인증키 / 필수값
    'searchId': '05', # 호출서비스구분 / 필수값
    'type' : 'json', # 데이터타입
    #'pIndex': '', # 페이지위치 
    #'pSize': '', # 페이지당요청숫자
    'SIDO' : '46', # 시도코드 2자리 / 필수값
    'SIGUN' : '170', # 시군구코드 3자리 / 필수값
    'QUERY': 'ALL' # 주파수구분 - D1~D10, A1~A5, ALL
}

response = requests.get(url, params=params, headers=headers)
json_str = response.content.decode('utf-8')
json_object = json.loads(json_str)
df = pd.DataFrame(json_object['RESULT'])

In [35]:
# 컬럼명 변경 (기존 컬럼명 -> 새로운 컬럼명)
df = df.rename(columns={
    'CUS_FLN': '시설자명',
    'FRQ_HZ': '주파수(MHz)',
    'RNUM': '번호',
    'RDS_PMS_NO': '허가번호',
    'RDS_CLL_NM': '호출명칭',
    'EMDG_CD': '전파형식코드',
    'LON': '경도',
    'RDS_TRS_ADR': '주소(설치장소)',
    'LAT': '위도'
})

# 컬럼 순서 변경 (배치 순서대로 변경)
new_column_order = [
    '허가번호',        # 1
    '호출명칭',        # 2
    '시설자명',        # 3
    '전파형식코드',    # 4
    '주파수(MHz)',          # 5
    '주소(설치장소)',  # 6
    '경도',            # 7
    '위도'             # 8
]

# 데이터프레임을 새로운 컬럼 순서로 재정렬
df = df[new_column_order]

In [40]:
df['위도좌표'] = df['위도'].apply(parse_dms)
df['경도좌표'] = df['경도'].apply(parse_dms)

# ======================================

### 2. 웹에서 직접 받은 후 가공 - O
##### 1) 웹에서 시도별 마을방송 무선국 데이터를 받습니다
##### 2) 파일을 병합합니다
##### 3) 데이터를 가공합니다 (전처리, preprocessing)

In [26]:
# 시도별 마을방송 무선국 파일이 들어있는 장소
folder_path = '원본데이터'

# 엑셀 파일만 필터링
excel_files = [f for f in os.listdir(folder_path) if f.endswith('.xls') or f.endswith('.xlsx')]

# 모든 엑셀 파일을 읽어서 하나의 데이터프레임으로 합치기
df_list = []  # 여러 데이터프레임을 저장할 리스트

# 각 엑셀 파일을 판다스로 읽어들여 리스트에 추가
for file in excel_files:
    file_path = os.path.join(folder_path, file)  # 파일 경로 완성
    df = pd.read_excel(file_path)  # 엑셀 파일을 읽어서 DataFrame으로 변환
    df_list.append(df)  # 읽은 DataFrame을 리스트에 추가

# 모든 데이터프레임을 하나로 합치기 (기본적으로 행 단위로 합침)
df = pd.concat(df_list, ignore_index=True)

In [28]:
# 위경도좌표 십진수변환
df['위도'] = df['위도'].apply(parse_dms)
df['경도'] = df['경도'].apply(parse_dms)

In [30]:
# '설치장소' 컬럼에서 >> '시도' '시군구' 추출
def split_address(address):
    try:
        words = address.split()  # 공백을 기준으로 단어를 분리
        
        # words가 3개 이상인 경우만 처리
        if len(words) >= 3:
            # 3번째 단어 (인덱스 2) 검사: 끝이 '시', '군', '구'가 아니면 None
            if not (words[2].endswith('시') or words[2].endswith('군') or words[2].endswith('구')):
                words[2] = None
        
        # 반환값: '시도', '시군구1', '시군구2' (세 번째 단어는 None일 수 있음)
        return pd.Series([words[0], words[1], words[2] if len(words) > 2 else None])
    
    except Exception as e:
        print(f"Error processing address: {address} \nError: {e}")
        # 결과가 제대로 추출되지 않았으면 None으로 반환
        return pd.Series([None, None, None])

In [32]:
df[['시도', '시군구1', '시군구2']] = df['설치장소'].apply(split_address)

Error processing address: nan 
Error: 'float' object has no attribute 'split'
Error processing address: nan 
Error: 'float' object has no attribute 'split'


In [34]:
df.drop_duplicates(subset='허가번호', inplace=True)

In [80]:
df['시군구'] = df['시군구1'].fillna('').astype(str) + " " + df['시군구2'].fillna('').astype(str)

In [86]:
# 불필요한 컬럼 제거
df = df.drop(['순번', '시군구1', '시군구2'], axis = 1)

In [88]:
df.columns

Index(['허가번호', '무선국 호출명', '시설자명', '전파형식코드', '주파수', '설치장소', '위도', '경도', '시도',
       '시군구'],
      dtype='object')

In [92]:
df = df[['허가번호', '무선국 호출명', '시설자명', '전파형식코드', '주파수', '설치장소',
'시도', '시군구', '위도', '경도']]